# Mapping GO IDs to GO Slim Terms

This code utilizes pipelines from the [paper-gonad-meth repo](https://github.com/epigeneticstoocean/paper-gonad-meth/tree/master) and [bioinformatics pipeline](https://robertslab.github.io/resources/bio-Annotation/#map-go-ids-to-goslims), both out of the Roberts Lab.

In [7]:
# loading packages
library(tidyverse)
library(dplyr)
library(GSEABase)
library(GO.db)

input file includes column of gene ID and column of Gene Ontology (GO) ID, separated by semi-colon

Gene Onology file for *Crassostrea virginica* obtained from [NCBI](https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/002/022/765/GCF_002022765.2_C_virginica-3.0/)

In [26]:
# Read the GAF file into R
GOobo <- read.table('/project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/Cvirginica_genome/GCF_002022765.2_C_virginica-3.0_gene_ontology.gaf', 
                          sep = "\t", 
                          header = FALSE, 
                          comment.char = "!", 
                          stringsAsFactors = FALSE)

# assign column names
colnames(GOobo) <- c('DB', 'GeneID', 'gene_id', 'Qualifier', 'Gene.Ontology.IDs', 'Reference', 'Evidence_Code', 'With,From', 'Aspect', 'Gene_Name', 'Gene_Synonym', 'Type', 'Taxon', 'Date', 'Assigned_By')

# selecting only the LOC number and GO ID columns
gene2GO <- GOobo[,c(3,5)]

# Aggregate GO IDs for each gene symbol
gene2GO <- aggregate(Gene.Ontology.IDs ~ gene_id, data = gene2GO, function(x) paste(unique(x), collapse = "; "))

# preview df
head(gene2GO)


,gene_id,Gene.Ontology.IDs
,<chr>,<chr>
1,LOC111099029,GO:0005261; GO:0098655
2,LOC111099032,GO:0004930; GO:0005886; GO:0007186
3,LOC111099033,GO:0016020; GO:0022857; GO:0055085
4,LOC111099034,GO:0005515
5,LOC111099036,GO:0008028; GO:0015718; GO:0055085
6,LOC111099039,GO:0004930; GO:0005886; GO:0007189


In [27]:
write.csv(gene2GO, '/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_MethylRAD_analysis_2018/analysis/DMGs_analysis/gene2GO.csv', row.names=FALSE)

In [30]:
head(read.csv('/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_MethylRAD_analysis_2018/analysis/DMGs_analysis/gene2GO.csv'))

,gene_id,Gene.Ontology.IDs
,<chr>,<chr>
1,LOC111099029,GO:0005261; GO:0098655
2,LOC111099032,GO:0004930; GO:0005886; GO:0007186
3,LOC111099033,GO:0016020; GO:0022857; GO:0055085
4,LOC111099034,GO:0005515
5,LOC111099036,GO:0008028; GO:0015718; GO:0055085
6,LOC111099039,GO:0004930; GO:0005886; GO:0007189


In [73]:
# copied from workflow pipeline linked above

# set variables

# Column names corresponding to gene name/ID and GO IDs
GO.ID.column <- "Gene.Ontology.IDs"
gene.ID.column <- "gene_id"

# Relative path or URL to input file
input.file <- "/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_MethylRAD_analysis_2018/analysis/DMGs_analysis/gene2GO.csv"


##### Official GO info - no need to change #####
goslims_obo <- "goslim_generic.obo"
goslims_url <- "http://current.geneontology.org/ontology/subsets/goslim_generic.obo"


In [71]:
# copied from workflow pipeline linked above

# download generic goslim-obo

# Find GSEAbase installation location
gseabase_location <- find.package("GSEABase")

# Load path to GOslim OBO file
goslim_obo_destintation <- file.path(gseabase_location, "extdata", goslims_obo, fsep = "/")

# Download the GOslim OBO file
download.file(url = goslims_url, destfile = goslim_obo_destintation)

# Loads package files
gseabase_files <- system.file("extdata", goslims_obo, package="GSEABase")

one unique gene per row, multiple GO IDs per gene

In [75]:
# copied from workflow pipeline linked above

# read in gene file

full.gene.df <- read.csv(file = input.file, header = TRUE)

str(full.gene.df)
head(full.gene.df)

'data.frame':	22614 obs. of  2 variables:
 $ gene_id          : chr  "LOC111099029" "LOC111099032" "LOC111099033" "LOC111099034" ...
 $ Gene.Ontology.IDs: chr  "GO:0005261; GO:0098655" "GO:0004930; GO:0005886; GO:0007186" "GO:0016020; GO:0022857; GO:0055085" "GO:0005515" ...


,gene_id,Gene.Ontology.IDs
,<chr>,<chr>
1,LOC111099029,GO:0005261; GO:0098655
2,LOC111099032,GO:0004930; GO:0005886; GO:0007186
3,LOC111099033,GO:0016020; GO:0022857; GO:0055085
4,LOC111099034,GO:0005515
5,LOC111099036,GO:0008028; GO:0015718; GO:0055085
6,LOC111099039,GO:0004930; GO:0005886; GO:0007189


In [76]:
# copied from workflow pipeline linked above

# remove NA and uniprotIDs

# Clean whitespace, filter NA/empty rows, select columns, and split GO terms using column name variables
library(dplyr) # for some reason I'm having conflicts with select()
gene.GO.df <- gene2GO %>%
  mutate(!!GO.ID.column := str_replace_all(.data[[GO.ID.column]], "\\s*;\\s*", ";")) %>%
  filter(!is.na(.data[[gene.ID.column]]) & !is.na(.data[[GO.ID.column]]) & .data[[GO.ID.column]] != "") %>%
  dplyr::select(all_of(c(gene.ID.column, GO.ID.column)))

head(gene.GO.df)

,gene_id,Gene.Ontology.IDs
,<chr>,<chr>
1,LOC111099029,GO:0005261;GO:0098655
2,LOC111099032,GO:0004930;GO:0005886;GO:0007186
3,LOC111099033,GO:0016020;GO:0022857;GO:0055085
4,LOC111099034,GO:0005515
5,LOC111099036,GO:0008028;GO:0015718;GO:0055085
6,LOC111099039,GO:0004930;GO:0005886;GO:0007189


flatten df so only one gene and one GO ID per row

In [78]:
# copied from workflow pipeline linked above

# flatten gene and GO IDs

flat.gene.GO.df <- gene.GO.df %>% separate_rows(!!sym(GO.ID.column), sep = ";")

str(flat.gene.GO.df)
head(flat.gene.GO.df)

tibble [77,015 × 2] (S3: tbl_df/tbl/data.frame)
 $ gene_id          : chr [1:77015] "LOC111099029" "LOC111099029" "LOC111099032" "LOC111099032" ...
 $ Gene.Ontology.IDs: chr [1:77015] "GO:0005261" "GO:0098655" "GO:0004930" "GO:0005886" ...


gene_id,Gene.Ontology.IDs
<chr>,<chr>
LOC111099029,GO:0005261
LOC111099029,GO:0098655
LOC111099032,GO:0004930
LOC111099032,GO:0005886
LOC111099032,GO:0007186
LOC111099033,GO:0016020


Grouping by GO ID - one unique GO ID per row, multiple genes per GO ID

In [79]:
# copied from workflow pipeline linked above

# group by GO

grouped.gene.GO.df <- flat.gene.GO.df %>%
  group_by(!!sym(GO.ID.column)) %>%
  summarise(!!gene.ID.column := paste(.data[[gene.ID.column]], collapse = ","))

str(grouped.gene.GO.df)
head(grouped.gene.GO.df)

tibble [5,357 × 2] (S3: tbl_df/tbl/data.frame)
 $ Gene.Ontology.IDs: chr [1:5357] "GO:0000002" "GO:0000003" "GO:0000009" "GO:0000012" ...
 $ gene_id          : chr [1:5357] "LOC111135903" "LOC111115224" "LOC111099485,LOC111099578,LOC111108038,LOC111119818" "LOC111111498,LOC111111868,LOC111128425,LOC111128516,LOC111135198,LOC111135354,LOC111136704" ...


Gene.Ontology.IDs,gene_id
<chr>,<chr>
GO:0000002,LOC111135903
GO:0000003,LOC111115224
GO:0000009,"LOC111099485,LOC111099578,LOC111108038,LOC111119818"
GO:0000012,"LOC111111498,LOC111111868,LOC111128425,LOC111128516,LOC111135198,LOC111135354,LOC111136704"
GO:0000014,"LOC111111711,LOC111111845,LOC111121059,LOC111126118"
GO:0000015,"LOC111132341,LOC111137133"


#### Mapping GO IDs to GO Slim

In [81]:
# copied from workflow pipeline linked above

# Vectorize GO IDs
go_ids <- grouped.gene.GO.df[[GO.ID.column]]

str(go_ids)
head(go_ids)

 chr [1:5357] "GO:0000002" "GO:0000003" "GO:0000009" "GO:0000012" ...


[1] "GO:0000002" "GO:0000003" "GO:0000009" "GO:0000012" "GO:0000014"
[6] "GO:0000015"

In [84]:
# copied from workflow pipeline linked above

# extract GOslims from OBO

# Create GSEAbase GOCollection using `go_ids`
myCollection <- GOCollection(go_ids)

# Retrieve GOslims from GO OBO file set
slim <- getOBOCollection(gseabase_files)

str(slim)

Formal class 'OBOCollection' [package "GSEABase"] with 7 slots
  ..@ .stanza     :'data.frame':	153 obs. of  1 variable:
  .. ..$ value: chr [1:153] "Root" "Term" "Term" "Term" ...
  ..@ .subset     :'data.frame':	22 obs. of  1 variable:
  .. ..$ value: chr [1:22] "Rhea list of ChEBI terms representing the major species at pH 7.3." "Term not to be used for direct annotation" "Terms planned for obsoletion" "AGR slim" ...
  ..@ .kv         :'data.frame':	2075 obs. of  3 variables:
  .. ..$ stanza_id: chr [1:2075] ".__Root__" ".__Root__" ".__Root__" ".__Root__" ...
  .. ..$ key      : chr [1:2075] "format-version" "data-version" "synonymtypedef" "synonymtypedef" ...
  .. ..$ value    : chr [1:2075] "1.2" "go/2024-11-03/subsets/goslim_generic.owl" "syngo_official_label \"label approved by the SynGO project\"" "systematic_synonym \"Systematic synonym\" EXACT" ...
  ..@ evidenceCode: chr [1:26] "EXP" "IDA" "IPI" "IMP" ...
  ..@ ontology    : chr NA
  ..@ ids         : chr [1:141] "GO:0000228

In [85]:
# copied from workflow pipeline linked above

# retrieve BP GOslims

# Retrieve Biological Process (BP) GOslims
slimdf <- goSlim(myCollection, slim, "BP", verbose)
str(slimdf)
head(slimdf)

'data.frame':	72 obs. of  3 variables:
 $ Count  : int  54 11 9 32 4 3 0 0 44 33 ...
 $ Percent: num  2.127 0.433 0.354 1.26 0.158 ...
 $ Term   : chr  "mitotic cell cycle" "cytokinesis" "cytoplasmic translation" "immune system process" ...


,Count,Percent,Term
,<int>,<dbl>,<chr>
GO:0000278,54,2.1268216,mitotic cell cycle
GO:0000910,11,0.4332414,cytokinesis
GO:0002181,9,0.3544703,cytoplasmic translation
GO:0002376,32,1.2603387,immune system process
GO:0003012,4,0.1575423,muscle system process
GO:0003013,3,0.1181568,circulatory system process


In [86]:
# copied from workflow pipeline linked above

#Map GO to GOslims

# List of GOslims and all GO IDs from `go_ids`
gomap <- as.list(GOBPOFFSPRING[rownames(slimdf)])

# Maps `go_ids` to matching GOslims
mapped <- lapply(gomap, intersect, ids(myCollection))

# Append all mapped GO IDs to `slimdf`
# `sapply` needed to apply paste() to create semi-colon delimited values
slimdf$GO.IDs <- sapply(lapply(gomap, intersect, ids(myCollection)), paste, collapse=";")

# Remove "character(0) string from "GO.IDs" column
slimdf$GO.IDs[slimdf$GO.IDs == "character(0)"] <- ""

# Add self-matching GOIDs to "GO.IDs" column, if not present
for (go_id in go_ids) {
  # Check if the go_id is present in the row names
  if (go_id %in% rownames(slimdf)) {
    # Check if the go_id is not present in the GO.IDs column
    # Also removes white space "trimws()" and converts all to upper case to handle
    # any weird, "invisible" formatting issues.
    if (!go_id %in% trimws(toupper(strsplit(slimdf[go_id, "GO.IDs"], ";")[[1]]))) {
      # Append the go_id to the GO.IDs column with a semi-colon separator
      if (length(slimdf$GO.IDs) > 0 && nchar(slimdf$GO.IDs[nrow(slimdf)]) > 0) {
        slimdf[go_id, "GO.IDs"] <- paste0(slimdf[go_id, "GO.IDs"], "; ", go_id)
      } else {
        slimdf[go_id, "GO.IDs"] <- go_id
      }
    }
  }
}

str(slimdf)
head(slimdf)

'data.frame':	72 obs. of  4 variables:
 $ Count  : int  54 11 9 32 4 3 0 0 44 33 ...
 $ Percent: num  2.127 0.433 0.354 1.26 0.158 ...
 $ Term   : chr  "mitotic cell cycle" "cytokinesis" "cytoplasmic translation" "immune system process" ...
 $ GO.IDs : chr  "GO:0000070;GO:0000082;GO:0000083;GO:0000086;GO:0000132;GO:0000281;GO:0007052;GO:0007076;GO:0007080;GO:0007088;G"| __truncated__ "GO:0000281;GO:0000915;GO:0032465;GO:0032467;GO:0036089;GO:0040038;GO:0061640;GO:0061952;GO:1902410;GO:1902412;GO:2000431" "GO:0001731;GO:0001732;GO:0002182;GO:0002183;GO:0002184;GO:0002188;GO:2000766;GO:2000767; GO:0002181" "GO:0001782;GO:0002218;GO:0002224;GO:0002250;GO:0002495;GO:0002682;GO:0002755;GO:0006955;GO:0006956;GO:0006959;G"| __truncated__ ...


,Count,Percent,Term,GO.IDs
,<int>,<dbl>,<chr>,<chr>
GO:0000278,54,2.1268216,mitotic cell cycle,GO:0000070;GO:0000082;GO:0000083;GO:0000086;GO:0000132;GO:0000281;GO:0007052;GO:0007076;GO:0007080;GO:0007088;GO:0007089;GO:0007091;GO:0007093;GO:0007094;GO:0007095;GO:0007096;GO:0007100;GO:0007346;GO:0010389;GO:0010971;GO:0010972;GO:0031573;GO:0033314;GO:0034087;GO:0040001;GO:0044772;GO:0044773;GO:0045840;GO:0045842;GO:0045930;GO:0051228;GO:0051256;GO:0051306;GO:0051315;GO:0060236;GO:0061780;GO:0061952;GO:0090266;GO:0090307;GO:1900087;GO:1901673;GO:1902410;GO:1902412;GO:1902969;GO:1902975;GO:1902977;GO:1902979;GO:1902983;GO:1903461;GO:1990426;GO:1990810;GO:2000045;GO:2000134; GO:0000278
GO:0000910,11,0.4332414,cytokinesis,GO:0000281;GO:0000915;GO:0032465;GO:0032467;GO:0036089;GO:0040038;GO:0061640;GO:0061952;GO:1902410;GO:1902412;GO:2000431
GO:0002181,9,0.3544703,cytoplasmic translation,GO:0001731;GO:0001732;GO:0002182;GO:0002183;GO:0002184;GO:0002188;GO:2000766;GO:2000767; GO:0002181
GO:0002376,32,1.2603387,immune system process,GO:0001782;GO:0002218;GO:0002224;GO:0002250;GO:0002495;GO:0002682;GO:0002755;GO:0006955;GO:0006956;GO:0006959;GO:0019731;GO:0030888;GO:0030890;GO:0031295;GO:0033151;GO:0034128;GO:0034164;GO:0039536;GO:0042102;GO:0042130;GO:0043029;GO:0045087;GO:0045088;GO:0045089;GO:0045579;GO:0046651;GO:0048821;GO:0050852;GO:0050901;GO:0051251;GO:0060335; GO:0002376
GO:0003012,4,0.1575423,muscle system process,GO:0003009;GO:0006936;GO:0006937;GO:0055117
GO:0003013,3,0.1181568,circulatory system process,GO:0003085;GO:0055117;GO:0060047


flatten GOslims file - one GO ID and one GOslim term per row

In [64]:
# copied from workflow pipeline linked above

# flatten-GOslims-file

# "Flatten" file so each row is single GO ID with corresponding GOslim
# rownames_to_column needed to retain row name info
slimdf_separated <- as.data.frame(slimdf %>%
  rownames_to_column('GOslim') %>%
  separate_rows(GO.IDs, sep = ";"))

# Group by unique GO ID
grouped_slimdf <- slimdf_separated %>%
  filter(!is.na(GO.IDs) & GO.IDs != "") %>%
  group_by(GO.IDs) %>%
  summarize(GOslim = paste(GOslim, collapse = ";"),
            Term = paste(Term, collapse = ";"))


str(grouped_slimdf)
head(grouped_slimdf)

tibble [1,850 × 3] (S3: tbl_df/tbl/data.frame)
 $ GO.IDs: chr [1:1850] " GO:0000278" " GO:0002181" " GO:0002376" " GO:0005975" ...
 $ GOslim: chr [1:1850] "GO:0000278" "GO:0002181" "GO:0002376" "GO:0005975" ...
 $ Term  : chr [1:1850] "mitotic cell cycle" "cytoplasmic translation" "immune system process" "carbohydrate metabolic process" ...


GO.IDs,GOslim,Term
<chr>,<chr>,<chr>
GO:0000278,GO:0000278,mitotic cell cycle
GO:0002181,GO:0002181,cytoplasmic translation
GO:0002376,GO:0002376,immune system process
GO:0005975,GO:0005975,carbohydrate metabolic process
GO:0006091,GO:0006091,generation of precursor metabolites and energy
GO:0006260,GO:0006260,DNA replication


In [65]:
# copied from workflow pipeline linked above

# sort and select slimdf counts

slimdf.sorted <- slimdf %>% arrange(desc(Count))

slim.count.df <- slimdf.sorted[,c(3,1)]
 # select(Term, Count)

str(slim.count.df)

head(slim.count.df)

'data.frame':	72 obs. of  2 variables:
 $ Term : chr  "signaling" "carbohydrate derivative metabolic process" "lipid metabolic process" "anatomical structure development" ...
 $ Count: int  232 151 133 126 119 98 96 94 78 77 ...


,Term,Count
,<chr>,<int>
GO:0023052,signaling,232
GO:1901135,carbohydrate derivative metabolic process,151
GO:0006629,lipid metabolic process,133
GO:0048856,anatomical structure development,126
GO:0055086,nucleobase-containing small molecule metabolic process,119
GO:0006520,amino acid metabolic process,98


here, count means out of the whole genome/GOslim file, the number of GO IDs that fall into that GOslim term


In [88]:
head(grouped_slimdf)
head(flat.gene.GO.df)

GO.IDs,GOslim,Term
<chr>,<chr>,<chr>
GO:0000278,GO:0000278,mitotic cell cycle
GO:0002181,GO:0002181,cytoplasmic translation
GO:0002376,GO:0002376,immune system process
GO:0005975,GO:0005975,carbohydrate metabolic process
GO:0006091,GO:0006091,generation of precursor metabolites and energy
GO:0006260,GO:0006260,DNA replication


gene_id,Gene.Ontology.IDs
<chr>,<chr>
LOC111099029,GO:0005261
LOC111099029,GO:0098655
LOC111099032,GO:0004930
LOC111099032,GO:0005886
LOC111099032,GO:0007186
LOC111099033,GO:0016020



so have two flat dfs - one that has one gene and one GO ID per row, and one with one GO ID and one GOslim term per row - want to merge them to get a new flat df with one gene and one GOslim term per row

In [92]:
# change column name to same as grouped_slimdf
colnames(flat.gene.GO.df) <- c('gene_id', 'GO.IDs')

gene2GOslim <- merge(grouped_slimdf, flat.gene.GO.df, by = 'GO.IDs')
head(gene2GOslim)

,GO.IDs,GOslim,Term,gene_id
,<chr>,<chr>,<chr>,<chr>
1,GO:0000002,GO:0007005,mitochondrion organization,LOC111135903
2,GO:0000012,GO:0006281,DNA repair,LOC111136704
3,GO:0000012,GO:0006281,DNA repair,LOC111111498
4,GO:0000012,GO:0006281,DNA repair,LOC111128516
5,GO:0000012,GO:0006281,DNA repair,LOC111135354
6,GO:0000012,GO:0006281,DNA repair,LOC111128425


so presumably, can take my list of significant genes and match to GO Slim terms?

following protocol from [paper-gonad-meth repo](https://github.com/epigeneticstoocean/paper-gonad-meth/tree/master), fitting [this pipeline](https://github.com/epigeneticstoocean/paper-gonad-meth/blob/master/code/14-Gene-Enrichment-with-GO-MWU.Rmd) to my GOSlim and GO-MWU files

all GO-MWU files were created from [GO-MWU.R](https://github.com/jgmcdonough/CE18_methylRAD_analysis/blob/master/analysis/DMGs_analysis/go_mwu/GO_MWU.R) - all **biological process** and using log2FoldChange as measure of significance


starting with:

## CH vs. HC


In [93]:
goMWU_ch <- read.csv('/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_MethylRAD_analysis_2018/analysis/DMGs_analysis/go_mwu/BP_allGenes_CH_lfc.csv', sep = '\t')

# change column name for gene to be the same in gene2GOslim
colnames(goMWU_ch) <- c('name', 'term', 'lev', 'gene_id', 'value')

head(goMWU_ch)

,name,term,lev,gene_id,value
,<chr>,<chr>,<int>,<chr>,<dbl>
1,supramolecular fiber organization,GO:0007015;GO:0097435,2,LOC111101101,-0.38722520
2,supramolecular fiber organization,GO:0007015;GO:0097435,2,LOC111108074,0.08059783
3,supramolecular fiber organization,GO:0007015;GO:0097435,2,LOC111109780,-1.04879576
4,supramolecular fiber organization,GO:0007015;GO:0097435,2,LOC111113129,-0.33325171
5,supramolecular fiber organization,GO:0007015;GO:0097435,2,LOC111113355,-0.28047982
6,supramolecular fiber organization,GO:0007015;GO:0097435,2,LOC111115662,-0.31195976


In [94]:
parentGOBPGOslim <- unique(merge(x = goMWU_ch, y = gene2GOslim, by = "gene_id", all.x = TRUE)) #Merge GO-MWU with GOslim information, retaining all GO-MWU entries. Remove any merging artifacts
head(parentGOBPGOslim) #Confirm merge
length(parentGOBPGOslim$GOslim) #Count number of entries

,gene_id,name,term,lev,value,GO.IDs,GOslim,Term
,<chr>,<chr>,<chr>,<int>,<dbl>,<chr>,<chr>,<chr>
1,LOC111099047,anion transport,GO:0071705;GO:0071702;GO:0006820,2,-0.6811806,GO:0003333,GO:0055085,transmembrane transport
2,LOC111099047,carboxylic acid transmembrane transport,GO:0003333;GO:0006865;GO:1905039;GO:0015849;GO:1903825;GO:0098656,4,-0.6811806,GO:0003333,GO:0055085,transmembrane transport
3,LOC111099047,localization,GO:0006810;GO:0051234;GO:0006811;GO:0051179,2,-0.6811806,GO:0003333,GO:0055085,transmembrane transport
4,LOC111099047,transmembrane transport,GO:0034220;GO:0055085,4,-0.6811806,GO:0003333,GO:0055085,transmembrane transport
5,LOC111099053,formation of translation preinitiation complex,GO:0001731,2,0.9973747,GO:0001731,GO:0002181;GO:0065003,cytoplasmic translation;protein-containing complex assembly
6,LOC111099053,formation of translation preinitiation complex,GO:0001731,2,0.9973747,GO:0001732,GO:0002181;GO:0065003,cytoplasmic translation;protein-containing complex assembly


[1] 10818

In [95]:
sum(is.na(parentGOBPGOslim$GOslim)) #Count how many entries do not have GOslim terms. 830/10818 GO-MWU entries do not have GOslim entries.

[1] 830

***NEED TO CHECK CODE BELOW*** - copied from paper-gonad-meth but need to check that it works for this 

In [100]:
parentGOBPGOslimCounts <- as.data.frame(table(parentGOBPGOslim$GOslim)) #Count the frequency of each GOslim term
colnames(parentGOBPGOslimCounts) <- c("GOslim", "frequency") #Rename columns
head(parentGOBPGOslimCounts) #Confirm counts

,GOslim,frequency
,<fct>,<int>
1,GO:0000278,194
2,GO:0000278;GO:0000910,36
3,GO:0000278;GO:0006260,25
4,GO:0000278;GO:0006260;GO:0065003,5
5,GO:0000278;GO:0006351;GO:0006355,30
6,GO:0000278;GO:0007010,9


In [101]:
parentGOBPGOslimCounts$percentage <- (parentGOBPGOslimCounts$frequency/sum(parentGOBPGOslimCounts$frequency)*100) #Create a new column with percentages of GOSlim terms instead of raw counts
head(parentGOBPGOslimCounts) #Confirm column creation
max(parentGOBPGOslimCounts$percentage)

,GOslim,frequency,percentage
,<fct>,<int>,<dbl>
1,GO:0000278,194,1.94233080
2,GO:0000278;GO:0000910,36,0.36043252
3,GO:0000278;GO:0006260,25,0.25030036
4,GO:0000278;GO:0006260;GO:0065003,5,0.05006007
5,GO:0000278;GO:0006351;GO:0006355,30,0.30036043
6,GO:0000278;GO:0007010,9,0.09010813


[1] 9.110933

In [98]:
parentGOBPGOslimCounts <- parentGOBPGOslimCounts[c(4, 1, 5, 11, 13,
                                                         2, 6, 3,
                                                         12,
                                                         7, 10, 9, 8),] #Reorganize rows based on broader functions. Rows are general activity, development, stress response, and metabolic processes
head(parentGOBPGOslimCounts) #Confirm organization

,CVGOslim,frequency,percentage
,<fct>,<int>,<dbl>
4,GO:0000278;GO:0006260;GO:0065003,5,0.05006007
1,GO:0000278,194,1.94233080
5,GO:0000278;GO:0006351;GO:0006355,30,0.30036043
11,GO:0000278;GO:0023052,71,0.71085302
13,GO:0000910,11,0.11013216
2,GO:0000278;GO:0000910,36,0.36043252


In [99]:
parentGOBPGOslimCounts$GOSlim <- c("Cell-Cell Signaling", "Cell Adhesion", "Death", "Signal Transduction", "Transport",
                                   "Cell Cycle and Proliferation", "Developmental Processes", "Cell Organization and Biogenesis",
                                   "Stress Response",
                                   "DNA Metabolism", "RNA Metabolism", "Protein Metabolism", "Other Metabolic Processes") #Capitalize labels
head(parentGOBPGOslimCounts) #Confirm capitalization

,CVGOslim,frequency,percentage,GOSlim
,<fct>,<int>,<dbl>,<chr>
4,GO:0000278;GO:0006260;GO:0065003,5,0.05006007,Cell-Cell Signaling
1,GO:0000278,194,1.94233080,Cell Adhesion
5,GO:0000278;GO:0006351;GO:0006355,30,0.30036043,Death
11,GO:0000278;GO:0023052,71,0.71085302,Signal Transduction
13,GO:0000910,11,0.11013216,Transport
2,GO:0000278;GO:0000910,36,0.36043252,Cell Cycle and Proliferation
